# Projeto 2 - Ciência dos Dados

Nome: Ellen Beatriz Shen

Nome: Gabriela Moreno Boriero

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***@_ellenbs_***

In [17]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [21]:
#Produto escolhido:
produto = 'rainbow six siege'

#Quantidade mínima de mensagens capturadas:
n = 500

#Quantidade mínima de mensagens para a base de treinamento:
t = 700

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'en'

Capturando os dados do twitter:

In [22]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [23]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [25]:
r6_raw = pd.read_excel("rainbow six siege.xlsx")

In [29]:
r6 = set(r6_raw.Treinamento)

In [30]:
r6_ = pd.Series(list(r6))

In [31]:
r6_.to_excel("base_r6.xlsx")

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [3]:
base_acai = pd.read_excel("base_acai.xlsx")

In [4]:
acai_gosta = base_acai[base_acai.Classe == 0]
acai_indiferente = base_acai[base_acai.Classe == 1]
acai_nao_gosta = base_acai[base_acai.Classe == 2]

In [5]:
series_gosta = pd.Series(list(acai_gosta.Treinamento))
series_indiferente = pd.Series(list(acai_indiferente.Treinamento))
series_nao_gosta = pd.Series(list(acai_nao_gosta.Treinamento))

In [6]:
gosta = []
for tweets in series_gosta:
    gosta.append(tweets)
gosta_texto=', '.join(gosta)

with open('gosta.txt','w', encoding="utf8") as arquivo:
    arquivo.write(gosta_texto)

In [7]:
indiferente = []
for tweets in series_indiferente:
    gosta.append(tweets)
gosta_texto=', '.join(gosta)

with open('indiferente.txt','w', encoding="utf8") as arquivo:
    arquivo.write(gosta_texto)

In [8]:
nao_gosta = []
for tweets in series_nao_gosta:
    gosta.append(tweets)
gosta_texto=', '.join(gosta)

with open('nao_gosta.txt','w', encoding="utf8") as arquivo:
    arquivo.write(gosta_texto)

In [9]:
import re 

def cleanup(text):
    #Função de limpeza muito simples que troca alguns sinais básicos por espaços
    import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed    

In [16]:
gosta_de_acai_raw = open("gosta.txt", "r", encoding="utf8").read()
indiferente_acai_raw = open("indiferente.txt", "r", encoding="utf8").read()
nao_gosta_de_acai_raw = open("nao_gosta.txt", "r", encoding="utf8").read()

In [11]:
gosta_de_acai = cleanup(gosta_de_acai_raw.lower())
indiferente_acai = cleanup(indiferente_acai_raw.lower())
nao_gosta_de_acai = cleanup(nao_gosta_de_acai_raw.lower())

In [12]:
serie_gosta = pd.Series(gosta_de_acai.split())
serie_indiferente = pd.Series(indiferente_acai.split())
serie_nao_gosta = pd.Series(nao_gosta_de_acai.split())

In [13]:
tabela_gosta = serie_gosta.value_counts()
tabela_indiferente = serie_indiferente.value_counts()
tabela_nao_gosta = serie_nao_gosta.value_counts()

In [14]:
tabela_gosta_acai_relativa = serie_gosta.value_counts(True)
tabela_indiferente_acai_relativa = serie_indiferente.value_counts(True)
tabela_nao_gosta_acai_relativa = serie_nao_gosta.value_counts(True)

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**